In [1]:
mypath = "AIS Nachtrag"

import h5py
import os
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
from openpyxl import Workbook
import datetime
today = datetime.date.today()

onlydirs = [f for f in listdir(mypath) if isdir(join(mypath, f))]

samplelist = {}

In [2]:
onlydirs

['AIS M1_WKL_1_#20-1-1-1',
 'AIS M1_WKL_1_#20-1-2-1',
 'AIS M1_WKL_1_#20-1-3-1',
 'AIS M1_WKL_1_#20-2-1-1',
 'AIS M1_WKL_1_#20-2-2-1',
 'AIS M1_WKL_1_#20-2-3-1',
 'AIS M1_WKL_1_#21-1-1-1',
 'AIS M1_WKL_1_#21-1-2-1',
 'AIS M1_WKL_1_#21-1-3-1',
 'AIS M1_WKL_1_#21-2-1-1',
 'AIS M1_WKL_1_#21-2-1-2',
 'AIS M1_WKL_1_#21-2-2-1',
 'AIS M1_WKL_1_#24-1-1-1',
 'AIS M1_WKL_1_#24-1-2-1',
 'AIS M1_WKL_1_#24-1-3-1',
 'AIS M1_WKL_1_#24-2-1-1',
 'AIS M1_WKL_1_#24-2-2-1',
 'AIS M1_WKL_1_#24-2-3-1',
 'AIS M1_WKL_1_#28-1-1-1',
 'AIS M1_WKL_1_#28-1-2-1',
 'AIS M1_WKL_1_#28-1-3-1',
 'AIS M1_WKL_1_#28-2-1-1',
 'AIS M1_WKL_1_#28-2-2-1',
 'AIS M1_WKL_1_#28-2-2-2',
 'AIS M1_WKL_1_#29-1-1-1',
 'AIS M1_WKL_1_#29-1-2-1',
 'AIS M1_WKL_1_#29-1-3-1',
 'AIS M1_WKL_1_#29-2-1-1',
 'AIS M1_WKL_1_#29-2-2-1',
 'AIS M1_WKL_1_#29-2-3-1',
 'AIS M1_WKL_1_#32-1-1-1',
 'AIS M1_WKL_1_#32-1-2-1',
 'AIS M1_WKL_1_#32-1-3-1',
 'AIS M1_WKL_1_#32-2-1-1',
 'AIS M1_WKL_1_#32-2-2-1',
 'AIS M1_WKL_1_#32-2-3-1',
 'AIS M1_WKL_1_#34-1-1-1',
 

In [3]:
h5filelist = []
for root, subdirs, files in os.walk(mypath):
    h5files = [join(root,f) for f in files if f.endswith(".h5")]
    h5filelist += h5files

In [4]:
h5filelist

['AIS Nachtrag\\AIS M1_WKL_1_#20-1-1-1\\M1_WKL_1_#20-1-1-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#20-1-2-1\\M1_WKL_1_#20-1-2-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#20-1-3-1\\M1_WKL_1_#20-1-3-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#20-2-1-1\\M1_WKL_1_#20-2-1-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#20-2-2-1\\M1_WKL_1_#20-2-2-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#20-2-3-1\\M1_WKL_1_#20-2-3-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#21-1-1-1\\M1_WKL_1_#21-1-1-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#21-1-2-1\\M1_WKL_1_#21-1-2-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#21-1-3-1\\M1_WKL_1_#21-1-3-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#21-2-1-1\\M1_WKL_1_#21-2-1-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#21-2-1-2\\M1_WKL_1_#21-2-1-2.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#21-2-2-1\\M1_WKL_1_#21-2-2-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#24-1-1-1\\M1_WKL_1_#24-1-1-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#24-1-2-1\\M1_WKL_1_#24-1-2-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#24-1-3-1\\M1_WKL_1_#24-1-3-1.h5',
 'AIS Nachtrag\\AIS M1_WKL_1_#24-2-1-1\\

In [5]:
wb = Workbook()
ws = wb.active
count = 0 
normallist = {}
axotomiertlist = {}
samplelist = {} 
for h5file in h5filelist:
    if h5py.is_hdf5(h5file):
        lala = h5py.File(h5file,"r")
        
        try:
            rois = [key for key in lala["Data"].keys()]
            lengths = []
            for roi in rois:
                length = lala["Data/"+roi+"/Physical"].attrs["AISPhysicalLength"]
                # length = lala["Data/"+roi+"/Pixeldata"].attrs["AISend"] - lala["Data/"+roi+"/Pixeldata"].attrs[
                # "AISstart"] 
                lengths.append(length*1024)
        
            lengths = np.array(lengths)
            seriesname = lala.attrs["seriesname"].split("_")[3].split("-")[0] # should evaluate to the animal
            print(seriesname)
            dirname = h5file.split("_")[3]
            print(dirname)
            axotomiert = dirname.split("-")[-1][0] #should evaluate to the A character
            print(axotomiert)
            if axotomiert =="A":
                axotomiertlist[seriesname] = np.append(axotomiertlist.get(seriesname,np.array([])),lengths)
            elif axotomiert =="N": 
                normallist[seriesname] = np.append(normallist.get(seriesname,np.array([])),lengths)
            else:
                samplelist[seriesname] = np.append(samplelist.get(seriesname,np.array([])),lengths)
            
            print(lala.attrs["seriesname"] + "    " +str(lengths.mean()) + "   " + str(np.std(lengths)))
        except KeyError as e :
            print("File {0} doesnt hava data Object: {1}".format(h5file,e))
            pass
        
    else:
        print("File " + h5file + " doesn't conform with h5 standards")
        pass
    
    count = 1
    if count > 5: break

#20
#20-1-1-1\M1
1
M1_WKL_1_#20-1-1-1    30.8911606823   6.32204490507
#20
#20-1-2-1\M1
1
M1_WKL_1_#20-1-2-1    30.9263042098   4.47670642339
#20
#20-1-3-1\M1
1
M1_WKL_1_#20-1-3-1    33.2838491804   7.1563451372
#20
#20-2-1-1\M1
1
M1_WKL_1_#20-2-1-1    34.0892216859   5.26447740071
#20
#20-2-2-1\M1
1
M1_WKL_1_#20-2-2-1    35.2753157394   4.67781176749
#20
#20-2-3-1\M1
1
M1_WKL_1_#20-2-3-1    34.0540781583   3.8500211574
#21
#21-1-1-1\M1
1
M1_WKL_1_#21-1-1-1    30.1685218979   4.58187899092
#21
#21-1-2-1\M1
1
M1_WKL_1_#21-1-2-1    34.5285157797   3.9487627828
#21
#21-1-3-1\M1
1
M1_WKL_1_#21-1-3-1    34.2446642114   4.49028507593
#21
#21-2-1-1\M1
1
M1_WKL_1_#21-2-1-1    29.6403705895   3.25805863402
#21
#21-2-1-2\M1
2
M1_WKL_1_#21-2-1-2    31.5742629983   1.77747939295
#21
#21-2-2-1\M1
1
M1_WKL_1_#21-2-2-1    34.3235118693   6.55744437074
#24
#24-1-1-1\M1
1
M1_WKL_1_#24-1-1-1    30.7370698309   4.63416061207
#24
#24-1-2-1\M1
1
M1_WKL_1_#24-1-2-1    34.3088687328   3.77230614483
#24
#24-1

In [6]:
for key,value in normallist.items():
    print("Sample: {0}  Mean: {1} Std: {2} AIS: {3}".format(key,value.mean(),np.std(value),len(value)))
    

In [7]:
for key,value in axotomiertlist.items():
    print("Sample: {0}  Mean: {1} Std: {2} AIS: {3}".format(key,value.mean(),np.std(value),len(value)))

In [8]:
for key,value in samplelist.items():
    print("Sample: {0}  Mean: {1} Std: {2} AIS: {3}".format(key,value.mean(),np.std(value),len(value)))

Sample: #29  Mean: 34.744169244014294 Std: 5.929238179780893 AIS: 55
Sample: #21  Mean: 32.300096648918114 Std: 4.712024792765694 AIS: 66
Sample: #32  Mean: 31.825613817147225 Std: 6.138197481824102 AIS: 53
Sample: #28  Mean: 34.532983177306086 Std: 4.937803231740051 AIS: 59
Sample: #20  Mean: 33.11534047155087 Std: 5.585539826429525 AIS: 65
Sample: #24  Mean: 33.107085604511845 Std: 5.626012089158624 AIS: 56
Sample: #34  Mean: 30.061588467134982 Std: 5.689656127899112 AIS: 57
Sample: #36  Mean: 30.3533119134664 Std: 5.943020067927084 AIS: 69


In [38]:
wb = Workbook()
ws = wb.active

for key,value in normallist.items():
    ws = wb.create_sheet("Sample {0}".format(key[1:]))
    ws.append(["AIS Lengths"])
    for item in value:
        ws.append([item])
    
wb.save("Silas nicht axotomiert.xlsx")


In [39]:
wb = Workbook()
ws = wb.active

for key,value in axotomiertlist.items():
    ws = wb.create_sheet("Sample {0}".format(key[1:]))
    ws.append(["AIS Lengths"])
    for item in value:
        ws.append([item])
    
wb.save("Silas axotomiert.xlsx")

In [9]:
wb = Workbook()
ws = wb.active

for key,value in samplelist.items():
    ws = wb.create_sheet("Sample {0}".format(key[1:]))
    ws.append(["AIS Lengths"])
    for item in value:
        ws.append([item])
    
wb.save("Silas unsortiert.xlsx")

In [25]:
print("T-Wert: {0} P-Wert: {1}".format(tvalue,pvalue))

T-Wert: 4.8303691249 P-Wert: 0.0000021309
